# Sequence Record

Essentially a sequence with database metadata attached to it. Depending on the source, this metadata can be surprisingly rich.

In [1]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
record =SeqRecord(Seq("MKQHKAMIVALIVICITAVVAALVTRKD",
 IUPAC.protein),
 id="YP_025292.1", name="HokC",
 description="toxic membrane protein, small")
print record

ID: YP_025292.1
Name: HokC
Description: toxic membrane protein, small
Number of features: 0
Seq('MKQHKAMIVALIVICITAVVAALVTRKD', IUPACProtein())


You get a `SeqRecord` when you:
- parse a file with sequence data in one of the standard formats
- obtain sequence data from an online resource (eg NCBI, UniProt, etc)

# Entrez Utilities - Sequence queries

Online access to NCBI's data. The [Bio.Entrez](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html) module makes use of the Entrez
Programming Utilities (also known as EUtils), described in detail [here at NCBI](http://www.ncbi.nlm.nih.gov/entrez/utils/). You usually get an XML output, which is parsed using some utility function in `Bio.Entrez`.

**Important:** Be aware that the Entrez utilities impose usage limits and you can get yourself blacklisted if you violate those. BioPython takes care of keeping an eye on those for you. You also should fill in a non-fake email address. See the [Frequency section](http://www.ncbi.nlm.nih.gov/books/NBK25497/#chapter2\2e Frequency_Timing_and_Registrati) in the manual.

# esearch

`esearch` searches and retrieves primary IDs (for use in EFetch, ELink, and ESummary) and term translations and optionally retains results for future use in the user's environment. `efetch` can then be used to download the actual records.


In [2]:
from Bio import Entrez
Entrez.email = "my_little_self@utu.fi"
handle = Entrez.esearch(db="nucleotide", retmax=10, term="opuntia[ORGN] accD")
print "Handle:", handle #A file-like object, you can read it
content=handle.read()
print "\n\nContent:", content

Handle: <addinfourl at 55810816 whose fp = <socket._fileobject object at 0x353d450>>


Content: <?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "http://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">
<eSearchResult><Count>3</Count><RetMax>3</RetMax><RetStart>0</RetStart><IdList>
<Id>377580661</Id>
<Id>156535673</Id>
<Id>156535671</Id>
</IdList><TranslationSet><Translation>     <From>opuntia[ORGN]</From>     <To>"Opuntia"[Organism]</To>    </Translation></TranslationSet><TranslationStack>   <TermSet>    <Term>"Opuntia"[Organism]</Term>    <Field>Organism</Field>    <Count>1725</Count>    <Explode>Y</Explode>   </TermSet>   <TermSet>    <Term>accD[All Fields]</Term>    <Field>All Fields</Field>    <Count>161140</Count>    <Explode>N</Explode>   </TermSet>   <OP>AND</OP>  </TranslationStack><QueryTranslation>"Opuntia"[Organism] AND accD[All Fields]</QueryTranslation></eSearchResult>



That's XML allright. Let's parse it. The primary method to do this is `Entrez.read()` but it expects a file. We have already consumed our handle - we read the data from it to the end so that it cannot be used again. We could query again, or we can use `StringIO` which is a module which can simulate file objects on top of strings.

In [3]:
import StringIO #Useful whenever you have your data in a string but a function which expects a file
content_filelike=StringIO.StringIO(content)
equery_result=Entrez.read(content_filelike)
print "Parsed result of equery:", equery_result

Parsed result of equery: {u'Count': '3', u'RetMax': '3', u'IdList': ['377580661', '156535673', '156535671'], u'TranslationStack': [{u'Count': '1725', u'Field': 'Organism', u'Term': '"Opuntia"[Organism]', u'Explode': 'Y'}, {u'Count': '161140', u'Field': 'All Fields', u'Term': 'accD[All Fields]', u'Explode': 'N'}, 'AND'], u'TranslationSet': [{u'To': '"Opuntia"[Organism]', u'From': 'opuntia[ORGN]'}], u'RetStart': '0', u'QueryTranslation': '"Opuntia"[Organism] AND accD[All Fields]'}


So we are having a dictionary. Most important here is the `IdList` key which holds the IDs which fit our query. Now we can fetch those.

### read() vs. parse()

Throughout BioPython in the different modules the function `read()` is used to parse an output which only has a single record (of whatever kind) and it returns that record plus it may even throw an error if the output seems to contain more than one record. On the other hand `parse()` is used to parse an output which has several records and returns a generator over them (ie something you can for-loop over once, or turn into a list with `list()`).

## efetch

Now that we have our IDs, we can fetch the records:

In [4]:
handle=Entrez.efetch("nucleotide",id=equery_result['IdList'],retmode="xml")
efetch_result=Entrez.parse(handle)
print "Efetch result", efetch_result #This gives a "generator". You may want to make this into a list, or loop over it
for r in efetch_result:
    print r

Efetch result <generator object parse at 0x363a8c0>
{u'GBSeq_moltype': 'DNA', u'GBSeq_source': 'chloroplast Opuntia decumbens', u'GBSeq_sequence': 'gagaccgggttacctgaagctattcaaacaggtataggtgaactaaacggtattcctttagcaattggtgttatggattttgggtttatagggggttctatgggagccgtagtaggtgaaaaaatcacccggttgattgaatatgctaccaaaaattcactaccccttattatagtatgtgcttctggaggggcacggatgcaagaaggcatcgtgagcttaatgcaaatggctaaaatatcttctgccttgcacgactatcactataaaccggtcgaaaaaaaattattatatatatcaattcttgcatcgcccactagtggtggtgtgacagccagttttggtatgttgggggatattattattgccgaaccagacgccgaaattgcgtttgcgggtaaaagagtaattgaggaagttttgaaggaagaagtacccgaaggtgcacaaacaaccgaaaatttattcgaaaatggtttattcgatccagtcttaccacgtaatcttttaaagagcgctctaagtgagttacttgagctgcacggtttttttcctttaaataaaacccaagccaag', u'GBSeq_primary-accession': 'HQ620723', u'GBSeq_definition': 'Opuntia decumbens voucher Martinez & Eggli 146a (ZSS) acetyl-CoA carboxylase beta subunit (accD) gene, partial cds; chloroplast', u'GBSeq_accession-version': 'HQ620723.1', u'GBSeq_topology': 'linear', u'GBSeq_le

This is **not** what we wanted! What we want is a `SeqRecord` object. Reason: `Entrez.parse()` is a generic function for parsing Entrez XML. We are receiving nucleotide database records, ie sequences. For this, we want [Bio.SeqIO](http://biopython.org/DIST/docs/_api_159/Bio.SeqIO-module.html) which is specialized in parsing the various sequence formats we can meet and in this particular case the *GenBank* format (`gb`  is what we want).

In [5]:
import Bio.SeqIO as SeqIO
handle=Entrez.efetch("nucleotide",id=equery_result['IdList'],rettype="gb") #Note: this now says "gb" to get the right format
efetch_result=SeqIO.parse(handle,"gb") #Note: this used to read Entrez.parse(handle)
print "Efetch result", efetch_result #This gives a "generator". You may want to make this into a list, or loop over it
for r in efetch_result:
    print repr(r)
    print
    print

Efetch result <generator object parse at 0x380b410>
SeqRecord(seq=Seq('GAGACCGGGTTACCTGAAGCTATTCAAACAGGTATAGGTGAACTAAACGGTATT...AAG', IUPACAmbiguousDNA()), id='HQ620723.1', name='HQ620723', description='Opuntia decumbens voucher Martinez & Eggli 146a (ZSS) acetyl-CoA carboxylase beta subunit (accD) gene, partial cds; chloroplast.', dbxrefs=[])


SeqRecord(seq=Seq('GGTGAAAAAATCACCCGGTTGATTGAATATGCTACCAAAAATTCACTACCCCTT...AAA', IUPACAmbiguousDNA()), id='EF590893.1', name='EF590893', description='Opuntia humifusa voucher NMNH 06-8355 acetyl-CoA carboxylase-D (accD) gene, partial cds; chloroplast.', dbxrefs=[])


SeqRecord(seq=Seq('GGTGAAAAAATCACCCGGTTGATTGAATATGCTACCAAAAATTCACTACCCCTT...AAA', IUPACAmbiguousDNA()), id='EF590892.1', name='EF590892', description='Opuntia fragilis voucher USBG s.n. acetyl-CoA carboxylase-D (accD) gene, partial cds; chloroplast.', dbxrefs=[])




Much better. Now we have SeqRecords. Let us summarize everything into a nice piece of code.

In [6]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "my_little_self@utu.fi"

def fetch_records(query):
    search_result_handle=Entrez.esearch(db="nucleotide", retmax=10, term=query) #Query Entrez
    search_result=Entrez.read(search_result_handle) #Parse the XML data you get in reply
    efetch_result_handle=Entrez.efetch(db="nucleotide", id=search_result['IdList'],rettype="gb") #Now fetch the actual sequences in GenBank format
    seq_records=SeqIO.parse(efetch_result_handle,format="gb") #Now parse the obtained data using SeqIO
    return seq_records

for seq_rec in fetch_records("opuntia[ORGN] accD"):
    print repr(seq_rec)
    


SeqRecord(seq=Seq('GAGACCGGGTTACCTGAAGCTATTCAAACAGGTATAGGTGAACTAAACGGTATT...AAG', IUPACAmbiguousDNA()), id='HQ620723.1', name='HQ620723', description='Opuntia decumbens voucher Martinez & Eggli 146a (ZSS) acetyl-CoA carboxylase beta subunit (accD) gene, partial cds; chloroplast.', dbxrefs=[])
SeqRecord(seq=Seq('GGTGAAAAAATCACCCGGTTGATTGAATATGCTACCAAAAATTCACTACCCCTT...AAA', IUPACAmbiguousDNA()), id='EF590893.1', name='EF590893', description='Opuntia humifusa voucher NMNH 06-8355 acetyl-CoA carboxylase-D (accD) gene, partial cds; chloroplast.', dbxrefs=[])
SeqRecord(seq=Seq('GGTGAAAAAATCACCCGGTTGATTGAATATGCTACCAAAAATTCACTACCCCTT...AAA', IUPACAmbiguousDNA()), id='EF590892.1', name='EF590892', description='Opuntia fragilis voucher USBG s.n. acetyl-CoA carboxylase-D (accD) gene, partial cds; chloroplast.', dbxrefs=[])


## Search & download recap (1)

- `Entrez.esearch()` to get a reply with list of IDs
- `Entrez.efetch()` to grab the actual records
- `SeqIO.parse()` to obtain a sequence of SeqRecord objects

This is okay for simple small queries but is highly discouraged for large queries. NCBI expects you to use the *query history* functionality which lets them bind your fetch to a previous search and use their internal caching mechanisms. Turn it on with `usehistory="y"` like such:

In [7]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "my_little_self@utu.fi"

search_result_handle=Entrez.esearch(db="nucleotide", retmax=10, term="opuntia[ORGN] accD", usehistory="y") #Query Entrez with history on
search_result=Entrez.read(search_result_handle) #Parse the XML data you get in reply
print "Search_result:", search_result


Search_result: {u'Count': '3', u'RetMax': '3', u'IdList': ['377580661', '156535673', '156535671'], u'TranslationStack': [{u'Count': '1725', u'Field': 'Organism', u'Term': '"Opuntia"[Organism]', u'Explode': 'Y'}, {u'Count': '161140', u'Field': 'All Fields', u'Term': 'accD[All Fields]', u'Explode': 'N'}, 'AND'], u'QueryTranslation': '"Opuntia"[Organism] AND accD[All Fields]', u'TranslationSet': [{u'To': '"Opuntia"[Organism]', u'From': 'opuntia[ORGN]'}], u'RetStart': '0', u'QueryKey': '1', u'WebEnv': 'NCID_1_938917986_130.14.18.34_9001_1452614215_1196749894_0MetA0_S_MegaStore_F_1'}


Note the `QueryKey` and `WebEnv` keys. This is the information which identifies your result set at NCBI's server and can be used to fetch the records like so:

In [8]:
efetch_result_handle=Entrez.efetch(db="nucleotide",
                                   query_key=search_result["QueryKey"],
                                   webenv=search_result["WebEnv"],
                                   rettype="gb") #Now fetch the actual sequences in GenBank format. Note how I don't give the IDs!
seq_records=SeqIO.parse(efetch_result_handle,format="gb") #Now parse the obtained data using SeqIO
for r in seq_records:
    print repr(r)
    print
    print
    

SeqRecord(seq=Seq('GAGACCGGGTTACCTGAAGCTATTCAAACAGGTATAGGTGAACTAAACGGTATT...AAG', IUPACAmbiguousDNA()), id='HQ620723.1', name='HQ620723', description='Opuntia decumbens voucher Martinez & Eggli 146a (ZSS) acetyl-CoA carboxylase beta subunit (accD) gene, partial cds; chloroplast.', dbxrefs=[])


SeqRecord(seq=Seq('GGTGAAAAAATCACCCGGTTGATTGAATATGCTACCAAAAATTCACTACCCCTT...AAA', IUPACAmbiguousDNA()), id='EF590893.1', name='EF590893', description='Opuntia humifusa voucher NMNH 06-8355 acetyl-CoA carboxylase-D (accD) gene, partial cds; chloroplast.', dbxrefs=[])


SeqRecord(seq=Seq('GGTGAAAAAATCACCCGGTTGATTGAATATGCTACCAAAAATTCACTACCCCTT...AAA', IUPACAmbiguousDNA()), id='EF590892.1', name='EF590892', description='Opuntia fragilis voucher USBG s.n. acetyl-CoA carboxylase-D (accD) gene, partial cds; chloroplast.', dbxrefs=[])




## Search & download recap (2)

- `Entrez.esearch(..., usehistory="y")` to get a reply and webenv + query_key
- `Entrez.efetch(..., webenv=, query_key=)` to grab the actual records from the previous search
- `SeqIO.parse()` to obtain a sequence of SeqRecord objects

Great! Now our problem is that if we have a large result set, thousands of IDs, Entrez doesn't let us grab them all at once with a single efetch. We need to get them bit-by-bit. Here's a complete example.

In [9]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "my_little_self@utu.fi"

def fetch_records(query, ret_max=10, fetch_batch=5):
    search_result_handle=Entrez.esearch(db="nucleotide", retmax=ret_max, term=query, usehistory="y") #Query Entrez
    search_result=Entrez.read(search_result_handle) #Parse the XML data I get in reply
    #The only thing you care about is Count, and the Webenv+QueryKey data
    all_records=[] #This will be our result list
    #Divide into blocks of fetch_batch results, download one at a time
    for start_index in range(0,len(search_result["IdList"]),fetch_batch):
        efetch_result_handle=Entrez.efetch(db="nucleotide", 
                                           retstart=start_index,
                                           retmax=fetch_batch,
                                           webenv=search_result["WebEnv"],
                                           query_key=search_result["QueryKey"],
                                           rettype="gb") #Now fetch the actual sequences in GenBank format
        #add these records into our list
        all_records.extend(SeqIO.parse(efetch_result_handle,format="gb")) #Now parse the obtained data using SeqIO
    return all_records

fetched_records_p53=fetch_records("p53", ret_max=50, fetch_batch=10) #get max 50 records, fetch 10 at a time
print "Fetched", len(fetched_records_p53), "records"
print "First few:", fetched_records_p53[:5]


Fetched 50 records
First few: [SeqRecord(seq=Seq('GTCATATCCCAGTGCTGACTCCCGGGCGTGCAGACCGCTAACTAGCTCACTCGC...AAA', IUPACAmbiguousDNA()), id='NM_001318470.1', name='NM_001318470', description='Homo sapiens TSC22 domain family member 3 (TSC22D3), transcript variant 5, mRNA.', dbxrefs=[]), SeqRecord(seq=Seq('GTCATATCCCAGTGCTGACTCCCGGGCGTGCAGACCGCTAACTAGCTCACTCGC...AAA', IUPACAmbiguousDNA()), id='NM_001318468.1', name='NM_001318468', description='Homo sapiens TSC22 domain family member 3 (TSC22D3), transcript variant 4, mRNA.', dbxrefs=[]), SeqRecord(seq=UnknownSeq(5132365, alphabet = IUPACAmbiguousDNA(), character = 'N'), id='NZ_LN600312.1', name='NZ_LN600312', description='Xanthomonas citri pv. citri genome assembly ASX_S_PRJEB7150_v1, scaffold ASX_S_XAC908_scaffold1, whole genome shotgun sequence.', dbxrefs=['BioProject:PRJNA224116 BioSample:SAMEA2752228 Assembly:GCF_001498875.1']), SeqRecord(seq=UnknownSeq(113613, alphabet = IUPACAmbiguousDNA(), character = 'N'), id='NZ_LJJL01000015.1', 

## Search & download - final version

- `Entrez.esearch(..., retmax=, usehistory="y")` to get a reply and webenv + query_key, searches for retmax records
- `for batch_idx in range(0, total_num_of_results, download_batch)` to chop the results into download_batch sized chunks
- `Entrez.efetch(retstart=, retmax=download_batch, webenv=, query_key=)` to grab one batch at a time
- `SeqIO.parse()` to obtain a sequence of SeqRecord objects for every batch

Let's grab a protein and see what the record has:

In [10]:
tp53_rec=SeqIO.read(Entrez.efetch(db="protein",id=["NP_000537"],rettype="gb"),"gb")
print repr(tp53_rec)


SeqRecord(seq=Seq('MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWF...DSD', IUPACProtein()), id='NP_000537.3', name='NP_000537', description='cellular tumor antigen p53 isoform a [Homo sapiens].', dbxrefs=[])


In [11]:
dir(tp53_rec)

['__add__',
 '__bool__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__getitem__',
 '__hash__',
 '__init__',
 '__iter__',
 '__len__',
 '__module__',
 '__new__',
 '__nonzero__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_per_letter_annotations',
 '_seq',
 '_set_per_letter_annotations',
 '_set_seq',
 'annotations',
 'dbxrefs',
 'description',
 'features',
 'format',
 'id',
 'letter_annotations',
 'lower',
 'name',
 'reverse_complement',
 'seq',
 'upper']

In [12]:
print "annotations:", dir(tp53_rec.annotations)
print "annotations['accessions']", tp53_rec.annotations['accessions']
print "annotations"

annotations: ['__class__', '__cmp__', '__contains__', '__delattr__', '__delitem__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'has_key', 'items', 'iteritems', 'iterkeys', 'itervalues', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values', 'viewitems', 'viewkeys', 'viewvalues']
annotations['accessions'] ['NP_000537']
annotations
